In [2]:
batch_size = 16
num_epochs = 10
learning_rate = 2e-4
S = 100

img_size = 64
img_channels = 3

num_groups = 8
first_conv_channels = 64
channel_multiplier = [1, 2, 4, 8]
has_attention = [first_conv_channels * 4, first_conv_channels * 8]
num_res_blocks = 2  # Number of residual blocks

total_timesteps = 1_000

diffusion_config = {
    "beta_0": 0.0001,
    "beta_T": 0.02,
    "T": total_timesteps,
}

In [3]:
from torchvision.datasets import LFWPeople
import torchvision.transforms as transforms
import torch

pil_transformer = transforms.Compose([
    transforms.PILToTensor()
])

train_ds = LFWPeople('/content/drive/MyDrive', split='train', download=True,
                        image_set='original', transform=lambda pil: pil_transformer(pil.resize(
                            (img_size, img_size))) / 255)

test_ds = LFWPeople('/content/drive/MyDrive', split='test', download=True,
                       image_set='original', transform=lambda pil: pil_transformer(pil.resize(
                            (img_size, img_size))) / 255)

ds = torch.utils.data.ConcatDataset([train_ds, test_ds])
# ds = torch.utils.data.Subset(train_ds, list(range(30)))

print(len(ds))

Files already downloaded and verified
Files already downloaded and verified
13233


In [4]:
import math
import torch
import torch.nn as nn
from torchvision.datasets import LFWPeople
import torchvision.transforms as transforms
import torch

def get_timestep_embedding(timesteps, embedding_dim):
    """
    This matches the implementation in Denoising Diffusion Probabilistic Models:
    From Fairseq.
    Build sinusoidal embeddings.
    This matches the implementation in tensor2tensor, but differs slightly
    from the description in Section 3.5 of "Attention Is All You Need".
    """
    assert len(timesteps.shape) == 1

    half_dim = embedding_dim // 2
    emb = math.log(10000) / (half_dim - 1)
    emb = torch.exp(torch.arange(half_dim, dtype=torch.float32) * -emb)
    emb = emb.to(device=timesteps.device)
    emb = timesteps.float()[:, None] * emb[None, :]
    emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=1)

    if embedding_dim % 2 == 1:  # zero pad
        emb = torch.nn.functional.pad(emb, (0,1,0,0))
    return emb

def nonlinearity(x):
    # swish
    return x*torch.sigmoid(x)


def Normalize(in_channels):
    return torch.nn.GroupNorm(num_groups=num_groups, num_channels=in_channels, eps=1e-6, affine=True)


class Upsample(nn.Module):
    def __init__(self, in_channels, with_conv):
        super().__init__()
        self.with_conv = with_conv
        if self.with_conv:
            self.conv = torch.nn.Conv2d(in_channels,
                                        in_channels,
                                        kernel_size=3,
                                        stride=1,
                                        padding=1)

    def forward(self, x):
        x = torch.nn.functional.interpolate(x, scale_factor=2.0, mode="nearest")
        if self.with_conv:
            x = self.conv(x)
        return x


class Downsample(nn.Module):
    def __init__(self, in_channels, with_conv):
        super().__init__()
        self.with_conv = with_conv
        if self.with_conv:
            # no asymmetric padding in torch conv, must do it ourselves
            self.conv = torch.nn.Conv2d(in_channels,
                                        in_channels,
                                        kernel_size=3,
                                        stride=2,
                                        padding=0)

    def forward(self, x):
        if self.with_conv:
            pad = (0,1,0,1)
            x = torch.nn.functional.pad(x, pad, mode="constant", value=0)
            x = self.conv(x)
        else:
            x = torch.nn.functional.avg_pool2d(x, kernel_size=2, stride=2)
        return x


class ResnetBlock(nn.Module):
    def __init__(self, *, in_channels, out_channels=None, conv_shortcut=False,
                 dropout, temb_channels=512):
        super().__init__()
        self.in_channels = in_channels
        out_channels = in_channels if out_channels is None else out_channels
        self.out_channels = out_channels
        self.use_conv_shortcut = conv_shortcut

        self.norm1 = Normalize(in_channels)
        self.conv1 = torch.nn.Conv2d(in_channels,
                                     out_channels,
                                     kernel_size=3,
                                     stride=1,
                                     padding=1)
        self.temb_proj = torch.nn.Linear(temb_channels,
                                         out_channels)
        self.norm2 = Normalize(out_channels)
        self.dropout = torch.nn.Dropout(dropout)
        self.conv2 = torch.nn.Conv2d(out_channels,
                                     out_channels,
                                     kernel_size=3,
                                     stride=1,
                                     padding=1)
        if self.in_channels != self.out_channels:
            if self.use_conv_shortcut:
                self.conv_shortcut = torch.nn.Conv2d(in_channels,
                                                     out_channels,
                                                     kernel_size=3,
                                                     stride=1,
                                                     padding=1)
            else:
                self.nin_shortcut = torch.nn.Conv2d(in_channels,
                                                    out_channels,
                                                    kernel_size=1,
                                                    stride=1,
                                                    padding=0)

    def forward(self, x, temb):
        h = x
        h = self.norm1(h)
        h = nonlinearity(h)
        h = self.conv1(h)

        h = h + self.temb_proj(nonlinearity(temb))[:,:,None,None]

        h = self.norm2(h)
        h = nonlinearity(h)
        h = self.dropout(h)
        h = self.conv2(h)

        if self.in_channels != self.out_channels:
            if self.use_conv_shortcut:
                x = self.conv_shortcut(x)
            else:
                x = self.nin_shortcut(x)

        return x+h


class AttnBlock(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.in_channels = in_channels

        self.norm = Normalize(in_channels)
        self.q = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.k = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.v = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.proj_out = torch.nn.Conv2d(in_channels,
                                        in_channels,
                                        kernel_size=1,
                                        stride=1,
                                        padding=0)


    def forward(self, x):
        h_ = x
        h_ = self.norm(h_)
        q = self.q(h_)
        k = self.k(h_)
        v = self.v(h_)

        # compute attention
        b,c,h,w = q.shape
        q = q.reshape(b,c,h*w)
        q = q.permute(0,2,1)   # b,hw,c
        k = k.reshape(b,c,h*w) # b,c,hw
        w_ = torch.bmm(q,k)     # b,hw,hw    w[b,i,j]=sum_c q[b,i,c]k[b,c,j]
        w_ = w_ * (int(c)**(-0.5))
        w_ = torch.nn.functional.softmax(w_, dim=2)

        # attend to values
        v = v.reshape(b,c,h*w)
        w_ = w_.permute(0,2,1)   # b,hw,hw (first hw of k, second of q)
        h_ = torch.bmm(v,w_)     # b, c,hw (hw of q) h_[b,c,j] = sum_i v[b,c,i] w_[b,i,j]
        h_ = h_.reshape(b,c,h,w)

        h_ = self.proj_out(h_)

        return x+h_

class Model(nn.Module):
    def __init__(self, *, ch, out_ch, ch_mult=(1,2,4,8), num_res_blocks,
                 attn_resolutions, dropout=0.0, resamp_with_conv=True, in_channels,
                 resolution):
        """
        ch: Starting curve-down filters of first unit
        out_ch: Output channels
        num_res_blocks: number of residuals in each unit(width of each encode-decode unit)
        attn_resolutions: specify which resolution to apply attention
        in_channels: images input channels
        resolution: images resolution
        """

        super().__init__()
        self.ch = ch
        self.temb_ch = self.ch*4
        self.num_resolutions = len(ch_mult)
        self.num_res_blocks = num_res_blocks
        self.resolution = resolution
        self.in_channels = in_channels

        # timestep embedding
        self.temb = nn.Module()
        self.temb.dense = nn.ModuleList([
            torch.nn.Linear(self.ch,
                            self.temb_ch),
            torch.nn.Linear(self.temb_ch,
                            self.temb_ch),
        ])

        # downsampling
        self.conv_in = torch.nn.Conv2d(in_channels,
                                       self.ch,
                                       kernel_size=3,
                                       stride=1,
                                       padding=1)

        curr_res = resolution
        in_ch_mult = (1,)+ch_mult

        self.down = nn.ModuleList()
        for i_level in range(self.num_resolutions):
            block = nn.ModuleList()
            attn = nn.ModuleList()
            block_in = ch*in_ch_mult[i_level]
            block_out = ch*ch_mult[i_level]
            for i_block in range(self.num_res_blocks):
                block.append(ResnetBlock(in_channels=block_in,
                                         out_channels=block_out,
                                         temb_channels=self.temb_ch,
                                         dropout=dropout))
                block_in = block_out
                if curr_res in attn_resolutions:
                    attn.append(AttnBlock(block_in))
            down = nn.Module()
            down.block = block
            down.attn = attn
            if i_level != self.num_resolutions-1:
                down.downsample = Downsample(block_in, resamp_with_conv)
                curr_res = curr_res // 2
            self.down.append(down)

        # middle
        self.mid = nn.Module()
        self.mid.block_1 = ResnetBlock(in_channels=block_in,
                                       out_channels=block_in,
                                       temb_channels=self.temb_ch,
                                       dropout=dropout)
        self.mid.attn_1 = AttnBlock(block_in)
        self.mid.block_2 = ResnetBlock(in_channels=block_in,
                                       out_channels=block_in,
                                       temb_channels=self.temb_ch,
                                       dropout=dropout)

        # upsampling
        self.up = nn.ModuleList()

        for i_level in reversed(range(self.num_resolutions)):
            block = nn.ModuleList()
            attn = nn.ModuleList()
            block_out = ch*ch_mult[i_level]
            skip_in = ch*ch_mult[i_level]
            for i_block in range(self.num_res_blocks+1):
                if i_block == self.num_res_blocks:
                    skip_in = ch*in_ch_mult[i_level]
                block.append(ResnetBlock(in_channels=block_in+skip_in,
                                         out_channels=block_out,
                                         temb_channels=self.temb_ch,
                                         dropout=dropout))
                block_in = block_out
                if curr_res in attn_resolutions:
                    attn.append(AttnBlock(block_in))

            up = nn.Module()
            up.block = block
            up.attn = attn
            if i_level != 0:
                up.upsample = Upsample(block_in, resamp_with_conv)
                curr_res = curr_res * 2
            self.up.insert(0, up) # prepend to get consistent order

        # end
        self.norm_out = Normalize(block_in)
        self.conv_out = torch.nn.Conv2d(block_in,
                                        out_ch,
                                        kernel_size=3,
                                        stride=1,
                                        padding=1)


    def forward(self, x, t):
        assert x.shape[2] == x.shape[3] == self.resolution

        # timestep embedding
        temb = get_timestep_embedding(t, self.ch)
        temb = self.temb.dense[0](temb)
        temb = nonlinearity(temb)
        temb = self.temb.dense[1](temb)

        # downsampling
        hs = [self.conv_in(x)]
        for i_level in range(self.num_resolutions):
            for i_block in range(self.num_res_blocks):
                h = self.down[i_level].block[i_block](hs[-1], temb)
                if len(self.down[i_level].attn) > 0:
                    h = self.down[i_level].attn[i_block](h)
                hs.append(h)
            if i_level != self.num_resolutions-1:
                hs.append(self.down[i_level].downsample(hs[-1]))

        # middle
        h = hs[-1]
        h = self.mid.block_1(h, temb)
        h = self.mid.attn_1(h)
        h = self.mid.block_2(h, temb)

        # upsampling
        for i_level in reversed(range(self.num_resolutions)):
            for i_block in range(self.num_res_blocks+1):
                h = self.up[i_level].block[i_block](
                    torch.cat([h, hs.pop()], dim=1), temb)
                if len(self.up[i_level].attn) > 0:
                    h = self.up[i_level].attn[i_block](h)
            if i_level != 0:
                h = self.up[i_level].upsample(h)

        # end
        h = self.norm_out(h)
        h = nonlinearity(h)
        h = self.conv_out(h)
        return h

In [5]:
import os
from tqdm import tqdm
import numpy as np
import torch
from torchvision.utils import save_image, make_grid

torch.manual_seed(0)
np.random.seed(0)

def _map_gpu(gpu):
    if gpu == 'cuda':
        return lambda x: x.cuda()
    else:
        return lambda x: x.to(torch.device('cuda:'+gpu))

map_gpu = _map_gpu('cuda')

def rescale(X, batch=True):
    if not batch:
        return (X - X.min()) / (X.max() - X.min())
    else:
        for i in range(X.shape[0]):
            X[i] = rescale(X[i], batch=False)
        return X


def std_normal(size):
    return map_gpu(torch.normal(0, 1, size=size))

def print_size(net):
    """
    Print the number of parameters of a network
    """
    if net is not None and isinstance(net, torch.nn.Module):
        module_parameters = filter(lambda p: p.requires_grad, net.parameters())
        params = sum([np.prod(p.size()) for p in module_parameters])
        print("{} Parameters: {:.6f}M".format(
            net.__class__.__name__, params / 1e6), flush=True)


def calc_diffusion_hyperparams(T, beta_0, beta_T):
    """
    Compute diffusion process hyperparameters

    Parameters:
    T (int):                    number of diffusion steps
    beta_0 and beta_T (float):  beta schedule start/end value,
                                where any beta_t in the middle is linearly interpolated

    Returns:
    a dictionary of diffusion hyperparameters including:
        T (int), Beta/Alpha/Alpha_bar/Sigma (torch.tensor on cpu, shape=(T, ))
    """

    Beta = torch.linspace(beta_0, beta_T, T)
    Alpha = 1 - Beta
    Alpha_bar = Alpha + 0
    Beta_tilde = Beta + 0
    for t in range(1, T):
        Alpha_bar[t] *= Alpha_bar[t-1]
        Beta_tilde[t] *= (1-Alpha_bar[t-1]) / (1-Alpha_bar[t])
    Sigma = torch.sqrt(Beta_tilde)

    _dh = {}
    _dh["T"], _dh["Beta"], _dh["Alpha"], _dh["Alpha_bar"], _dh["Sigma"] = T, Beta, Alpha, Alpha_bar, Sigma
    diffusion_hyperparams = _dh

    # print(Alpha_bar)
    return diffusion_hyperparams


def bisearch(f, domain, target, eps=1e-8):
    """
    find smallest x such that f(x) > target

    Parameters:
    f (function):               function
    domain (tuple):             x in (left, right)
    target (float):             target value

    Returns:
    x (float)
    """
    #
    sign = -1 if target < 0 else 1
    left, right = domain
    for _ in range(1000):
        x = (left + right) / 2
        if f(x) < target:
            right = x
        elif f(x) > (1 + sign * eps) * target:
            left = x
        else:
            break
    return x


def get_VAR_noise(S, schedule='linear'):
    """
    Compute VAR noise levels

    Parameters:
    S (int):            approximante diffusion process length
    schedule (str):     linear or quadratic

    Returns:
    np array of noise levels, size = (S, )
    """
    target = np.prod(1 - np.linspace(diffusion_config["beta_0"], diffusion_config["beta_T"], diffusion_config["T"]))

    if schedule == 'linear':
        g = lambda x: np.linspace(diffusion_config["beta_0"], x, S)
        domain = (diffusion_config["beta_0"], 0.99)
    elif schedule == 'quadratic':
        g = lambda x: np.array([diffusion_config["beta_0"] * (1+i*x) ** 2 for i in range(S)])
        domain = (0.0, 0.95 / np.sqrt(diffusion_config["beta_0"]) / S)
    else:
        raise NotImplementedError

    f = lambda x: np.prod(1 - g(x))
    largest_var = bisearch(f, domain, target, eps=1e-4)
    return g(largest_var)


def _log_gamma(x):
    # Gamma(x+1) ~= sqrt(2\pi x) * (x/e)^x  (1 + 1 / 12x)
    y = x - 1
    return np.log(2 * np.pi * y) / 2 + y * (np.log(y) - 1) + np.log(1 + 1 / (12 * y))


def _log_cont_noise(t, beta_0, beta_T, T):
    # We want log_cont_noise(t, beta_0, beta_T, T) ~= np.log(Alpha_bar[-1].numpy())
    delta_beta = (beta_T - beta_0) / (T - 1)
    _c = (1.0 - beta_0) / delta_beta
    t_1 = t + 1
    return t_1 * np.log(delta_beta) + _log_gamma(_c + 1) - _log_gamma(_c - t_1 + 1)

# VAR
def _precompute_VAR_steps(diffusion_hyperparams, user_defined_eta):
    _dh = diffusion_hyperparams
    T, Alpha, Alpha_bar, Beta = _dh["T"], _dh["Alpha"], _dh["Alpha_bar"], _dh["Beta"]
    assert len(Alpha_bar) == T

    # compute diffusion hyperparameters for user defined noise
    T_user = len(user_defined_eta)
    Beta_tilde = map_gpu(torch.from_numpy(user_defined_eta)).to(torch.float32)
    Gamma_bar = 1 - Beta_tilde
    for t in range(1, T_user):
        Gamma_bar[t] *= Gamma_bar[t-1]

    assert Gamma_bar[0] <= Alpha_bar[0] and Gamma_bar[-1] >= Alpha_bar[-1]

    continuous_steps = []
    with torch.no_grad():
        for t in range(T_user-1, -1, -1):
            t_adapted = None
            for i in range(T - 1):
                if Alpha_bar[i] >= Gamma_bar[t] > Alpha_bar[i+1]:
                    t_adapted = bisearch(f=lambda _t: _log_cont_noise(_t, Beta[0].cpu().numpy(), Beta[-1].cpu().numpy(), T),
                                            domain=(i-0.01, i+1.01),
                                            target=np.log(Gamma_bar[t].cpu().numpy()))
                    break
            if t_adapted is None:
                t_adapted = T - 1
            continuous_steps.append(t_adapted)  # must be decreasing
    return continuous_steps


def VAR_sampling(net, size, diffusion_hyperparams, user_defined_eta, kappa, continuous_steps):
    """
    Perform the complete sampling step according to user defined variances

    Parameters:
    net (torch network):            the model
    size (tuple):                   size of tensor to be generated,
                                    usually is (number of audios to generate, channels=1, length of audio)
    diffusion_hyperparams (dict):   dictionary of diffusion hyperparameters returned by calc_diffusion_hyperparams
                                    note, the tensors need to be cuda tensors
    user_defined_eta (np.array):    User defined noise
    kappa (float):                  factor multipled over sigma, between 0 and 1
    continuous_steps (list):        continuous steps computed from user_defined_eta

    Returns:
    the generated images in torch.tensor, shape=size
    """

    _dh = diffusion_hyperparams
    T, Alpha, Alpha_bar, Beta = _dh["T"], _dh["Alpha"], _dh["Alpha_bar"], _dh["Beta"]
    assert len(Alpha_bar) == T
    assert len(size) == 4
    assert 0.0 <= kappa <= 1.0

    # compute diffusion hyperparameters for user defined noise
    T_user = len(user_defined_eta)
    Beta_tilde = map_gpu(torch.from_numpy(user_defined_eta)).to(torch.float32)
    Gamma_bar = 1 - Beta_tilde
    for t in range(1, T_user):
        Gamma_bar[t] *= Gamma_bar[t-1]

    assert Gamma_bar[0] <= Alpha_bar[0] and Gamma_bar[-1] >= Alpha_bar[-1]

    # print('begin sampling, total number of reverse steps = %s' % T_user)

    x = std_normal(size)
    with torch.no_grad():
        for i, tau in enumerate(continuous_steps):
            diffusion_steps = tau * map_gpu(torch.ones(size[0]))
            epsilon_theta = net(x, diffusion_steps)
            if i == T_user - 1:  # the next step is to generate x_0
                assert abs(tau) < 0.1
                alpha_next = torch.tensor(1.0)
                sigma = torch.tensor(0.0)
            else:
                alpha_next = Gamma_bar[T_user-1-i - 1]
                sigma = kappa * torch.sqrt((1-alpha_next) / (1-Gamma_bar[T_user-1-i]) * (1 - Gamma_bar[T_user-1-i] / alpha_next))
            x *= torch.sqrt(alpha_next / Gamma_bar[T_user-1-i])
            c = torch.sqrt(1 - alpha_next - sigma ** 2) - torch.sqrt(1 - Gamma_bar[T_user-1-i]) * torch.sqrt(alpha_next / Gamma_bar[T_user-1-i])
            x += c * epsilon_theta + sigma * std_normal(size)

    return x


def generate(net, model_config,
             n_generate, batchsize,
             kappa=1.0,
             S=100,
             output_name=None):
    """
    Parameters:
    output_name (str):              save generated images to this folder
    net(object):                    torch model
    model_config (dic):             dic of model config
    n_generate (int):               number of generated samples
    batchsize (int):                batch size of training

    Returns:
    Generated images (tensor):      (B, C, H, W) where C = 3
    """
    if batchsize > n_generate:
        batchsize = n_generate
    assert n_generate % batchsize == 0

    if output_name is not None:
      if 'generated' not in os.listdir():
          os.mkdir('generated')
      if output_name not in os.listdir('generated'):
          os.mkdir(os.path.join('generated', output_name))

    # map diffusion hyperparameters to gpu
    diffusion_hyperparams = calc_diffusion_hyperparams(**diffusion_config)
    for key in diffusion_hyperparams:
        if key != "T":
            diffusion_hyperparams[key] = map_gpu(diffusion_hyperparams[key])

    print_size(net)

    # sampling
    C, H, W = model_config["in_channels"], model_config["resolution"], model_config["resolution"]

    user_defined_eta = get_VAR_noise(S)
    continuous_steps = _precompute_VAR_steps(diffusion_hyperparams, user_defined_eta)
    Xi = VAR_sampling(net, (batchsize, C, H, W),
                      diffusion_hyperparams,
                      user_defined_eta,
                      kappa=kappa,
                      continuous_steps=continuous_steps)

    if output_name is not None:
      # save image
      for j, x in enumerate(rescale(Xi)):
          index = j
          save_image(x, fp=os.path.join('generated', output_name, '{}.jpg'.format(index)))
      save_image(make_grid(rescale(Xi)[:64]), fp=os.path.join('generated', '{}.jpg'.format(output_name)))

    return Xi

In [6]:
import tensorflow as tf

    # "beta_0": 0.0001,
    # "beta_T": 0.02,
    # "T": total_timesteps,

class GaussianDiffusion:
    """Gaussian diffusion utility.

    Args:
        beta_start: Start value of the scheduled variance
        beta_end: End value of the scheduled variance
        timesteps: Number of time steps in the forward process
    """

    def __init__(
        self,
        beta_start=diffusion_config['beta_0'],
        beta_end=diffusion_config['beta_T'],
        timesteps=diffusion_config['T'],
        clip_min=-1.0,
        clip_max=1.0,
    ):
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.timesteps = timesteps
        self.clip_min = clip_min
        self.clip_max = clip_max

        # Define the linear variance schedule
        self.betas = betas = np.linspace(
            beta_start,
            beta_end,
            timesteps,
            dtype=np.float32,  # Using float64 for better precision
        )
        self.num_timesteps = int(timesteps)

        alphas = 1.0 - betas
        alphas_cumprod = np.cumprod(alphas, axis=0)
        alphas_cumprod_prev = np.append(1.0, alphas_cumprod[:-1])

        self.betas = tf.constant(betas, dtype=tf.float32)
        self.alphas_cumprod = tf.constant(alphas_cumprod, dtype=tf.float32)
        self.alphas_cumprod_prev = tf.constant(alphas_cumprod_prev, dtype=tf.float32)

        # Calculations for diffusion q(x_t | x_{t-1}) and others
        self.sqrt_alphas_cumprod = tf.constant(
            np.sqrt(alphas_cumprod), dtype=tf.float32
        )

        self.sqrt_one_minus_alphas_cumprod = tf.constant(
            np.sqrt(1.0 - alphas_cumprod), dtype=tf.float32
        )

        self.log_one_minus_alphas_cumprod = tf.constant(
            np.log(1.0 - alphas_cumprod), dtype=tf.float32
        )

        self.sqrt_recip_alphas_cumprod = tf.constant(
            np.sqrt(1.0 / alphas_cumprod), dtype=tf.float32
        )
        self.sqrt_recipm1_alphas_cumprod = tf.constant(
            np.sqrt(1.0 / alphas_cumprod - 1), dtype=tf.float32
        )

        # Calculations for posterior q(x_{t-1} | x_t, x_0)
        posterior_variance = (
            betas * (1.0 - alphas_cumprod_prev) / (1.0 - alphas_cumprod)
        )
        self.posterior_variance = tf.constant(posterior_variance, dtype=tf.float32)

        # Log calculation clipped because the posterior variance is 0 at the beginning
        # of the diffusion chain
        self.posterior_log_variance_clipped = tf.constant(
            np.log(np.maximum(posterior_variance, 1e-20)), dtype=tf.float32
        )

        self.posterior_mean_coef1 = tf.constant(
            betas * np.sqrt(alphas_cumprod_prev) / (1.0 - alphas_cumprod),
            dtype=tf.float32,
        )

        self.posterior_mean_coef2 = tf.constant(
            (1.0 - alphas_cumprod_prev) * np.sqrt(alphas) / (1.0 - alphas_cumprod),
            dtype=tf.float32,
        )

    def _extract(self, a, t, x_shape):
        """Extract some coefficients at specified timesteps,
        then reshape to [batch_size, 1, 1, 1, 1, ...] for broadcasting purposes.

        Args:
            a: Tensor to extract from
            t: Timestep for which the coefficients are to be extracted
            x_shape: Shape of the current batched samples
        """
        batch_size = x_shape[0]
        out = tf.gather(a, t)
        return tf.reshape(out, [batch_size, 1, 1, 1])

    def q_mean_variance(self, x_start, t):
        """Extracts the mean, and the variance at current timestep.

        Args:
            x_start: Initial sample (before the first diffusion step)
            t: Current timestep
        """
        x_start_shape = tf.shape(x_start)
        mean = self._extract(self.sqrt_alphas_cumprod, t, x_start_shape) * x_start
        variance = self._extract(1.0 - self.alphas_cumprod, t, x_start_shape)
        log_variance = self._extract(
            self.log_one_minus_alphas_cumprod, t, x_start_shape
        )
        return mean, variance, log_variance

    def q_sample(self, x_start, t, noise):
        """Diffuse the data.

        Args:
            x_start: Initial sample (before the first diffusion step)
            t: Current timestep
            noise: Gaussian noise to be added at the current timestep
        Returns:
            Diffused samples at timestep `t`
        """

        # B, C, H, W
        # B, H, W, C

        x_start_shape = tf.shape(x_start)
        result = (
            self._extract(self.sqrt_alphas_cumprod, t, tf.shape(x_start)) * x_start
            + self._extract(self.sqrt_one_minus_alphas_cumprod, t, x_start_shape)
            * noise
        )

        return result

    def predict_start_from_noise(self, x_t, t, noise):
        x_t_shape = tf.shape(x_t)
        return (
            self._extract(self.sqrt_recip_alphas_cumprod, t, x_t_shape) * x_t
            - self._extract(self.sqrt_recipm1_alphas_cumprod, t, x_t_shape) * noise
        )

    def q_posterior(self, x_start, x_t, t):
        """Compute the mean and variance of the diffusion
        posterior q(x_{t-1} | x_t, x_0).

        Args:
            x_start: Stating point(sample) for the posterior computation
            x_t: Sample at timestep `t`
            t: Current timestep
        Returns:
            Posterior mean and variance at current timestep
        """
        x_t_shape = tf.shape(x_t)
        posterior_mean = (
            self._extract(self.posterior_mean_coef1, t, x_t_shape) * x_start
            + self._extract(self.posterior_mean_coef2, t, x_t_shape) * x_t
        )
        posterior_variance = self._extract(self.posterior_variance, t, x_t_shape)
        posterior_log_variance_clipped = self._extract(
            self.posterior_log_variance_clipped, t, x_t_shape
        )
        return posterior_mean, posterior_variance, posterior_log_variance_clipped

    def p_mean_variance(self, pred_noise, x, t, clip_denoised=True):
        x_recon = self.predict_start_from_noise(x, t=t, noise=pred_noise)
        if clip_denoised:
            x_recon = tf.clip_by_value(x_recon, self.clip_min, self.clip_max)

        model_mean, posterior_variance, posterior_log_variance = self.q_posterior(
            x_start=x_recon, x_t=x, t=t
        )
        return model_mean, posterior_variance, posterior_log_variance

    def p_sample(self, pred_noise, x, t, clip_denoised=True):
        """Sample from the diffusion model.

        Args:
            pred_noise: Noise predicted by the diffusion model
            x: Samples at a given timestep for which the noise was predicted
            t: Current timestep
            clip_denoised (bool): Whether to clip the predicted noise
                within the specified range or not.
        """
        model_mean, _, model_log_variance = self.p_mean_variance(
            pred_noise, x=x, t=t, clip_denoised=clip_denoised
        )
        noise = tf.random.normal(shape=x.shape, dtype=x.dtype)
        # No noise when t == 0
        nonzero_mask = tf.reshape(
            1 - tf.cast(tf.equal(t, 0), tf.float32), [tf.shape(x)[0], 1, 1, 1]
        )
        return model_mean + nonzero_mask * tf.exp(0.5 * model_log_variance) * noise

In [7]:
def torch_to_tf_tensor(torch_tensor):
  np_format = torch_tensor.numpy()
  # print(np_format.dtype)
  return tf.convert_to_tensor(np_format)

def tf_to_torch_tensor(tf_tensor):
  np_format = tf_tensor.numpy()
  return torch.from_numpy(np_format)

# B, C, H, W -> torch
# B, H, W, C -> tf
def torch_to_tf_dim(torch_tensor):
  x = torch.swapaxes(torch_tensor, 1, 3)
  x = torch.swapaxes(x, 1, 2)
  return torch_to_tf_tensor(x)

def tf_to_torch_dim(tf_tensor):
  x = tf_to_torch_tensor(tf_tensor)
  x = torch.swapaxes(x, 1, 3)
  return torch.swapaxes(x, 2, 3)

In [7]:
# gdf_util = GaussianDiffusion(timesteps=total_timesteps)

In [8]:
# from torch.utils.tensorboard.writer import SummaryWriter
# from datetime import datetime
# from tqdm import tqdm
# import time

# class ModelTrainer():
#   def __init__(self, model, loss, optimizer, time_steps=100):
#     self.model = model
#     self.loss = loss
#     self.optimizer = optimizer
#     self.time_steps = time_steps

#   def fit(self, ds, epochs=1, callbacks=[], **kwargs):
#     print(f'Starting fitting with {self.time_steps}')

#     running_loss = 0.
#     last_loss = 0.

#     i = 0
#     data_size = len(ds.dataset)

#     timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
#     writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
#     epoch_number = 0

#     start = time.time()

#     for epoch in range(epochs):
#       print('EPOCH {}:'.format(epoch + 1))
#       model.train()

#       i = 0
#       bar = tqdm(ds)
#       for data in bar:
#         inputs, _ = data

#         current_batch_size = inputs.size()[0]

#         self.optimizer.zero_grad()
#         t = torch.Tensor.to(
#             torch.floor(torch.rand(size=(current_batch_size,)) * self.time_steps),
#             dtype=torch.int32)

#         noise = torch.randn(size=inputs.size())

#         inputs = torch_to_tf_dim(inputs)
#         tf_t = torch_to_tf_tensor(t)
#         tf_noise = torch_to_tf_dim(noise)

#         noised_t = gdf_util.q_sample(inputs, tf_t, tf_noise) # 3 torch tensors -> converted -> processed by tensorflow -> 1 torch tensor
#         noised_t = tf_to_torch_dim(noised_t)

#         outputs = self.model(noised_t.to('cuda'), t.to('cuda'))
#         loss = self.loss(outputs.to('cuda'), noise.to('cuda'))
#         loss.backward()

#         self.optimizer.step()
#         running_loss += loss.item()
#         if i % 16 == 15:
#             tb_x = epoch * current_batch_size + i + 1
#             writer.add_scalar('Loss/train', last_loss, tb_x)
#             last_loss = running_loss / current_batch_size # loss per batch
#             bar.set_description_str(f'batch {i + 1}/{data_size // current_batch_size} loss: {last_loss:.4f}')

#             running_loss = 0.

#         i += 1

#       for callback in callbacks:
#         callback(model)

#       print(f'Run for {time.time() - start}s after {epoch} epochs!')

#     return last_loss

In [8]:
from matplotlib import pyplot as plt
import cv2
from sklearn.preprocessing import MinMaxScaler

plot_counts = 16
num_rows = 2
num_cols = plot_counts // num_rows

model_config = {
    "in_channels": img_channels,
    "resolution": img_size
}

save_path = '/content/drive/MyDrive/FastDPM/model.torch'

def plot(net):
    images = generate(net,
             model_config,
             plot_counts,
             batch_size,
              S=100,
             kappa=1.0)

    _, ax = plt.subplots(num_rows, num_cols, figsize=(10, 10))

    print(f'Plotting... Approximation using S={S}')
    for i, image in enumerate(images):
        cpu_image = image.detach().cpu()
        # C, H, W
        cpu_image = torch.swapaxes(cpu_image, 0, 2)
        cpu_image = torch.swapaxes(cpu_image, 0, 1).numpy()
        ori_shape = cpu_image.shape

        cpu_image = MinMaxScaler().fit_transform(cpu_image.reshape((
            cpu_image.shape[0] * cpu_image.shape[1], cpu_image.shape[2]
        ))).reshape(ori_shape)

        if num_rows == 1:
            ax[i].imshow(cpu_image)
            ax[i].axis("off")
        else:
            ax[i // num_cols, i % num_cols].imshow(cpu_image)
            ax[i // num_cols, i % num_cols].axis("off")

    plt.tight_layout()
    plt.show()

def save_model(net):
  with open(save_path, 'wb') as f:
    torch.save(net.module.state_dict(), f)

In [10]:
# from torch.nn import MSELoss
# from torch.optim import Adam

# print('Training info')
# print(f'bs: {batch_size}')
# print(f'n_epochs: {num_epochs}')
# print(f'time steps: {total_timesteps}')
# print(f'lr = {learning_rate}')
# print(f'img_size: {img_size}')

# """
#   ch: Starting curve-down filters of first unit
#   out_ch: Output channels num
#   num_res_blocks: number of residuals in each unit(width of each encode-decode unit)
#   attn_resolutions: specify which resolution to apply attention
#   in_channels:  first conv filters
#   resolution: images resolution
# """

# loader = torch.utils.data.DataLoader(
#              ds,
#              batch_size=batch_size,
#              shuffle=True,
#              num_workers=112,
#              persistent_workers=True)

# model = Model(
#   ch=first_conv_channels,
#   out_ch=img_channels,
#   num_res_blocks=num_res_blocks,
#   attn_resolutions=has_attention,
#   in_channels=3,
#   resolution=img_size)

# model = nn.DataParallel(model)
# model = model.to('cuda')

# trainer = ModelTrainer(model, MSELoss(),
#                        Adam(model.parameters(),
#                             lr=learning_rate), time_steps=total_timesteps)

# # plot(model)
# trainer.fit(loader, epochs=6, callbacks=[plot, save_model])

In [9]:
loaded_model =  Model(
  ch=first_conv_channels,
  out_ch=img_channels,
  num_res_blocks=num_res_blocks,
  attn_resolutions=has_attention,
  in_channels=3,
  resolution=img_size)

loaded_model.load_state_dict(torch.load(save_path))

loaded = nn.DataParallel(loaded_model)
loaded = loaded.to('cuda')

cnt = 0
for outer in tqdm(range(int(math.ceil(10_000 // batch_size)))):
  images = generate(loaded,
             model_config,
             plot_counts,
             batch_size,
              S=100,
             kappa=1.0)
  for i, image in enumerate(images):
      cpu_image = image.detach().cpu()
      # C, H, W
      cpu_image = torch.swapaxes(cpu_image, 0, 2)
      cpu_image = torch.swapaxes(cpu_image, 0, 1).numpy()
      ori_shape = cpu_image.shape
      cpu_image = MinMaxScaler().fit_transform(cpu_image.reshape((
          cpu_image.shape[0] * cpu_image.shape[1], cpu_image.shape[2]
      ))).reshape(ori_shape)

      # print(cpu_image)

      cnt += 1

      cv2.imwrite(f'/content/drive/MyDrive/FastDPM/fake/{cnt}.png',
                  (cpu_image * 255).astype(np.uint8))

  if cnt == 10_000:
    break

# trainer = ModelTrainer(loaded, MSELoss(),
#                        Adam(loaded.parameters(),
#                             lr=learning_rate), time_steps=total_timesteps)

# loader = torch.utils.data.DataLoader(
#              ds,
#              batch_size=batch_size,
#              shuffle=True,
#              num_workers=56,
#              persistent_workers=True)
# trainer.fit(loader, epochs=15, callbacks=[plot, save_model])

  0%|          | 0/625 [00:00<?, ?it/s]

DataParallel Parameters: 56.574595M


  0%|          | 1/625 [00:47<8:13:10, 47.42s/it]

DataParallel Parameters: 56.574595M


  0%|          | 2/625 [01:03<5:00:49, 28.97s/it]

...

100%|█████████▉| 623/625 [2:25:15<00:21, 10.64s/it]

DataParallel Parameters: 56.574595M
DataParallel Parameters: 56.574595M


100%|█████████▉| 624/625 [2:25:26<00:10, 10.58s/it]

DataParallel Parameters: 56.574595M
DataParallel Parameters: 56.574595M


100%|█████████▉| 624/625 [2:25:37<00:14, 14.00s/it]

